# Advent of Code: Day 4

### --- Day 4: Giant Squid ---
You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

`
22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
 `
 
After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):

`22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7`
 
After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:

`
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
 `
 
Finally, 24 is drawn:

`
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
 `
 
At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?


### --- Part Two ---
On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

Figure out which board will win last. Once it wins, what would its final score be?

In [534]:
import numpy as np

def bingo_input(input_file:str):
    """Parse input bingo data - first two lines are called numbers, rest are bingo board arrays"""
    depths = []    
    with open(input_file, 'r') as file:
        # Read first line as the called numbers into list
        callnums = file.readline().strip().split(',')
        callnums = list(map(int, callnums))
        
        # Now create list of arrays for the boards
        bingo_boards = []
        current_board = []
        for line in file:
            if len(line) <= 1:
                if len(current_board) > 1:
                    bingo_boards.append(np.array(current_board).astype(int))
                current_board = []
            else:
                current_board.append(line.strip().split())
        # Catch last line
        bingo_boards.append(np.array(current_board).astype(int))

    return callnums, bingo_boards

In [550]:
class Bingo(object):
    """ 
    Contains bingo game object.  Main data structure exists in the following form:
    
    Data objects 
    
    callnums (list):    numbers called in order
    bingoboards (list): list of arrays of bingo boards
    scoreboards (list): list of arrays of 0 filled boards to be marked with 1 
                        when same index bingo board wins 
    win_conds (list):   list of arrays with vertical and horizontal 1s indicating a 
                        winning score
    win_status (list):  list of booleans representing whether equivalent bingo boards 
                        have won already
    current_num_idx (int): index for current round and called number
    
    """
    
    def __init__(self, bingoboards:list, callnums:list):
        """
        Args:
        
        bingoboards (list): list of arrays representing bingo boards
        callnums (list):    list of called numbers in order of round
        """
        self.bingoboards = bingoboards
        self.callnums = callnums
        self.current_num_idx = 0 
        self.scoreboards = self.score_board_arrays(len(bingoboards))
        self.win_conds = self.win_condition_arrays()
        self.win_status = self.win_status_list(len(bingoboards))
        self.winning_board = None
        self.winning_marks = None
        self.winning_marked_board = None
        self.winning_num_idx = None
        self.winners = []
        
    def win_condition_arrays(self):
        empty = np.full((5,5),0)
        wins = []
        for i in range(empty.shape[0]):
            win = np.copy(empty)
            win[:,i] = 1
            wins.append(win)
        for i in range(empty.shape[1]):
            win = np.copy(empty)
            win[i,:] = 1
            wins.append(win)
        return wins
    
    def score_board_arrays(self, num:int):
        score_boards = []
        for i in range(num):
            score_boards.append(np.full((5,5),0))
        return score_boards
    
    def win_status_list(self, num:int):
        win_status = []
        for i in range(num):
            win_status.append(False)
        return win_status
    
    def next_num(self):
        if self.current_num_idx < len(self.callnums)-1:
            self.current_num_idx += 1
        
    def callnum_find(self):
        current_num = self.callnums[self.current_num_idx]
        for i, board in enumerate(self.bingoboards):
            found = np.where(board == current_num)
            if found[0].size > 0:
                self.callnum_mark_score(i,found[0][0],found[1][0])
            
    def callnum_mark_score(self, board_idx, mark_0, mark_1):
        if self.win_status[board_idx] == False:
            self.scoreboards[board_idx][mark_0][mark_1] = 1
    
    def callnum_check_win(self):
        winners = []
        for i, score in enumerate(self.scoreboards):
            if self.win_status[i] == False:
                for win in self.win_conds:
                    cond = win*score
                    if (win*score==win).all():
                        self.win_status[i] = True
                        self.winning_num_idx = self.current_num_idx
                        winners.append(i)
        return winners               
            
    def find_all(self, mode='first'):
        for i in range(len(self.callnums)):
            print(f'calling number {self.callnums[i]}')
            self.callnum_find()
            board_nums = self.callnum_check_win()
            if len(board_nums) > 0:
                for board_num in board_nums:
                    print(f'We have a winner!  Board: {board_num}')
                    self.winning_board = self.bingoboards[board_num]
                    self.marks = self.scoreboards[board_num]
                    self.winners.append((callnums[i], board_num, self.bingoboards[board_num],self.scoreboards[board_num]))
                if mode == 'first':
                    break
                elif mode == 'last':
                    pass
                else:
                    raise ValueException('Mode must be "first" or "last"')
            self.next_num()
    
    def find_score(self):
        inverted = 1 - self.marks
        self.winning_marked_board = self.winning_board * inverted
        self.score = np.sum(self.winning_marked_board) * self.callnums[self.winning_num_idx]
        print(f'Winning board: \n{self.winning_board}')
        print(f'Marked board: \n{self.winning_marked_board}')
        print(f'Final score (unmarked sum) {np.sum(self.winning_marked_board)} * '\
              f'last called num {self.callnums[self.winning_num_idx]}')
        print(self.score)
        return self.score

In [551]:
# Workflow - part 1

# Input
callnums, bingoboards = bingo_input('data/d04_bingo.txt')

# Set up boards
bingo = Bingo(bingoboards, callnums)
# Go through call numbers, break at winning condition
bingo.find_all(mode='first')
# Determine winning score
bingo.find_score()

calling number 46
calling number 79
calling number 77
calling number 45
calling number 57
calling number 34
calling number 44
calling number 13
calling number 32
calling number 88
calling number 86
calling number 82
calling number 91
calling number 97
calling number 89
calling number 1
calling number 48
calling number 31
calling number 18
calling number 10
calling number 55
calling number 74
calling number 24
calling number 11
calling number 80
calling number 78
calling number 28
We have a winner!  Board: 60
Winning board: 
[[19 85 36 73 71]
 [65 62 14 52  3]
 [30 83 44 41  5]
 [55 15  0 61 95]
 [28 13 32 31 88]]
Marked board: 
[[19 85 36 73 71]
 [65 62 14 52  3]
 [30 83  0 41  5]
 [ 0 15  0 61 95]
 [ 0  0  0  0  0]]
Final score (unmarked sum) 810 * last called num 28
22680


22680

In [552]:
# Part2 identical but switch find_all mode to set winning board to last rather than first

callnums, bingoboards = bingo_input('data/d04_bingo.txt')

#Set up boards
bingo = Bingo(bingoboards, callnums)
# Go through call numbers, break at winning condition
bingo.find_all(mode='last')
# Determine winning score
bingo.find_score()

calling number 46
calling number 79
calling number 77
calling number 45
calling number 57
calling number 34
calling number 44
calling number 13
calling number 32
calling number 88
calling number 86
calling number 82
calling number 91
calling number 97
calling number 89
calling number 1
calling number 48
calling number 31
calling number 18
calling number 10
calling number 55
calling number 74
calling number 24
calling number 11
calling number 80
calling number 78
calling number 28
We have a winner!  Board: 60
calling number 37
We have a winner!  Board: 51
calling number 47
We have a winner!  Board: 68
calling number 17
calling number 21
We have a winner!  Board: 45
calling number 61
We have a winner!  Board: 71
calling number 26
calling number 85
We have a winner!  Board: 74
calling number 99
We have a winner!  Board: 31
We have a winner!  Board: 79
calling number 96
calling number 23
calling number 70
We have a winner!  Board: 2
calling number 3
We have a winner!  Board: 94
calling num

16168